### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy import plotting
import subjects


def get_jump_dist(arr):
    # arr = np.apply_along_axis(
    #         np.convolve, axis=0, arr=arr, v=np.ones(2 * 4 + 1), mode="same"
    #     )
    mean_jd = np.mean(np.abs(np.diff(np.argmax(arr, axis=0))))
    std_jd = np.std(np.abs(np.diff(np.argmax(arr, axis=0))))
    return mean_jd, std_jd

### Jump distance distribution SD vs NSD

In [ ]:
sessions = subjects.pf_sess()

jump_df = []
for sub, sess in enumerate(sessions):
    neurons = sess.neurons_stable.get_neuron_type(["pyr", "mua"])
    neurons = neurons[neurons.firing_rate <= 10]

    replay_pbe = sess.replay_radon_mua
    replay_pbe_df = replay_pbe.to_dataframe()
    metadata = replay_pbe.metadata
    starts = replay_pbe.starts

    pbe_filter = sess.pbe_filters.to_dataframe()
    # good_bool = pbe_filter.is_rpl & pbe_filter.is_5units & pbe_filter.is_rest

    up_posteriors = metadata["up_posterior"]
    down_posteriors = metadata["down_posterior"]

    up_score = replay_pbe_df["up_score"].values
    down_score = replay_pbe_df["down_score"].values

    wcorr = np.maximum(np.abs(up_score), np.abs(down_score))

    posteriors = [up_posteriors, down_posteriors]
    posteriors = [
        posteriors[np.argmax([up_score[i], down_score[i]])][i]
        for i in range(len(up_score))
    ]

    dx = 1 / posteriors[0].shape[0]
    # dx=2
    jump_dist = np.array([get_jump_dist(_)[0] for _ in posteriors]) * dx
    # jump_dist = jump_dist[good_bool]
    # starts = starts[good_bool]

    epochs = sess.get_zt_epochs()
    starts_bool, _, starts_labels = epochs.contains(starts)

    df = pd.DataFrame(
        dict(
            zt=starts_labels,
            score=jump_dist[starts_bool],
            n_neurons=neurons.n_neurons,
            method="jump",
            session=sub,
            name=sess.animal.name + sess.animal.day,
            grp=sess.tag,
        )
    )

    score_pre, score_maze = [
        (f := df.groupby("zt")).get_group("PRE").score.values,
        f.get_group("MAZE").score.values,
    ]

    perc_rel_pre = [
        stats.percentileofscore(score_pre, _, kind="strict") for _ in df.score
    ]

    df["score_rel_pre"] = df.score / score_pre.mean()
    df["score_rel_maze"] = df.score / score_maze.mean()
    df["perc_rel_pre"] = perc_rel_pre

    jump_df.append(df)

jump_df = pd.concat(jump_df, ignore_index=True)
subjects.GroupData().save(jump_df, "replay_jumpdist_mua")

In [ ]:
from plotters import violinplot

_, ax = plt.subplots(1, 1)
violinplot(data=jump_df, x="zt", y="score", stat_anot=True)
# sns.stripplot(data=df,x='zt',y='jump',hue='grp')

In [ ]:
_, axs = plt.subplots(4, 5, sharex="row", sharey="row")

nbins = 20
xbins = np.linspace(0, 300, nbins)
ybins = np.linspace(0, 1, nbins)

rw_df = jump_df
zts = rw_df.zt.unique()

for i, zt in enumerate(zts[:1]):
    df = rw_df[(rw_df.zt == zt)]

    # ax=axs[0,i]
    # sns.histplot(data=df,x='radon',y='wcorr',hue='grp',ax=ax)

    hist_pre = []
    for g, grp in enumerate(["NSD", "SD"]):
        x = df[df.grp == grp].jump.values
        y = df[df.grp == grp].wcorr.values

        score_hist = np.histogram2d(x, y, bins=[xbins, ybins])[0]
        hist_pre.append(score_hist / np.sum(score_hist))


for i, zt in enumerate(zts):
    df = rw_df[(rw_df.zt == zt)]

    # ax=axs[0,i]
    # sns.histplot(data=df,x='radon',y='wcorr',hue='grp',ax=ax)

    hist_all = []
    for g, grp in enumerate(["NSD", "SD"]):
        x = df[df.grp == grp].jump.values
        y = df[df.grp == grp].wcorr.values

        score_hist = np.histogram2d(x, y, bins=[xbins, ybins])[0]
        norm_hist = score_hist / np.sum(score_hist)
        hist_all.append(norm_hist)

        ax = axs[g, i]
        # hist_diff = hist_all[0]-hist_all[1]
        ax.pcolormesh(xbins, ybins, norm_hist, cmap="jet", vmin=-0.01, vmax=0.01)

        ax = axs[3, i]
        sns.regplot(
            x=x,
            y=y,
            color=subjects.colors_sd()[g],
            ax=ax,
            scatter=False,
            scatter_kws=dict(s=2),
        )

    axs[2, i].pcolormesh(
        xbins, ybins, hist_all[0] - hist_all[1], cmap="jet", vmin=-0.002, vmax=0.002
    )

    # ax=axs[3,i]
    # ax.scatter(x,y,s=2)

### Jump distance (speed) power law
- are jump distances more brownian for sleep deprivation vs regular sleep

In [ ]:
sessions = subjects.pf_sess()


def get_motion(arr):
    y = np.argmax(arr, axis=0) * 2
    t = np.arange(arr.shape[1]) * 0.02
    dt = np.abs(t[np.newaxis, :] - t[:, np.newaxis])
    dy = np.abs(y[np.newaxis, :] - y[:, np.newaxis])

    indx = np.tril_indices(dy.shape[0], k=-1)

    return dy[indx], dt[indx]


speed_df = []
for sub, sess in enumerate(sessions):
    starts = sess.replay_pbe.starts
    replay_pbe = sess.replay_wcorr_mua.to_dataframe()

    pbe_filter = sess.pbe_filters.to_dataframe()

    good_bool = (
        pbe_filter.is_rpl
        & pbe_filter.is_5neurons
        # & pbe_filter.is_lowtheta
        & pbe_filter.is_rest
    )

    metadata = sess.replay_wcorr_mua.metadata
    up_posteriors = metadata["up_posterior"]
    down_posteriors = metadata["down_posterior"]

    up_wcorr = replay_pbe["up_score"].values
    down_wcorr = -replay_pbe["down_score"].values

    posteriors = [up_posteriors, down_posteriors]
    posteriors = [
        posteriors[np.argmax([np.abs(up_wcorr[i]), np.abs(down_wcorr[i])])][i]
        for i in range(len(up_wcorr))
    ]

    motion_metric = motion_metric.T

    epochs = sess.get_zt_epochs()

    for i, e in enumerate(epochs.itertuples()):
        indx = (starts > e.start) & (starts < e.stop) & good_bool
        motion_metric = np.hstack(
            [np.vstack(get_motion(_)) for i, _ in enumerate(posteriors) if indx[i]]
        ).T
        df = pd.DataFrame(
            dict(
                zt=e.label,
                ddist=motion_metric[:, 0],
                dt=motion_metric[:, 1],
                session=sub,
                grp=sess.tag,
            )
        )

        speed_df.append(df)


speed_df = pd.concat(speed_df, ignore_index=True)

In [ ]:
_, axs = plt.subplots(1, 5, sharey=True)

zts = speed_df.zt.unique()

for i, zt in enumerate(zts):
    df = speed_df[speed_df.zt == zt]
    ax = axs[i]
    sns.lineplot(
        data=df,
        x="dt",
        y="ddist",
        hue="grp",
        ax=ax,
        palette=subjects.colors_sd(),
        err_style="bars",
    )

    slopes = []
    for g, grp in enumerate(["NSD", "SD"]):
        grp_df = df[df.grp == grp]
        t = grp_df.dt.values
        dy = grp_df.ddist.values

        bins = np.arange(0.001, 0.07, 0.02)
        mean_dist = stats.binned_statistic(t, dy, bins=bins)[0]
        bin_centers = bins[:-1] + 0.01

        linfit = stats.linregress(np.log10(bin_centers), np.log10(mean_dist))
        slopes.append(linfit.slope)

    ax.text(0.03, 150, f"{slopes[0].round(2),slopes[1].round(2)}")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.legend("", frameon=False)

In [ ]:
np.arange(0.001, 0.07, 0.02)

In [ ]:
_, ax = plt.subplots()
ax.plot(bin_centers, mean_dist, ".")